In [1]:
%matplotlib inline

import numpy as np

In [2]:
# helper for gradient checking
def approximate_gradient(x, func, eps=1e-5):
    res = np.zeros(x.size)
    
    for i in range(x.size):
        d = np.zeros(x.size)
        d[i] = eps
        
        res[i] = (func(x + d) - func(x - d)) / (2 * eps)
    
    return res

# Neural Networks

## Feed-Forward Network Functions

## Network Training

## Error Backpropagation

In [163]:
def ensure_node(obj):
    return obj if isinstance(obj, Node) else Constant(obj) 


class Node:
    def __add__(self, b):
        return Addition(self, ensure_node(b))
    
    def __radd__(self, b):
        return Addition(ensure_node(b), self)
    
    def __sub__(self, b):
        return Subtraction(self, ensure_node(b))

    def __rsub__(self, b):
        return Subtraction(ensure_node(b), self)

    def __mul__(self, b):
        return Multiplication(self, ensure_node(b))
    
    def __rmul__(self, b):
        return Multiplication(ensure_node(b), self)
    
    def __truediv__(self, b):
        return Division(self, ensure_node(b))
    
    def __rtruediv__(self, b):
        return Division(ensure_node(b), self)
    
    def __eq__(self, other):
        return (type(self) == type(other)) and self._key() == other._key()
    
    def __hash__(self):
        return hash((type(self),) + self._key())

    
class IdentityNode(Node):
    def __eq__(self, other):
        return id(self) == id(other)
    
    def __hash__(self):
        return hash(id(self))
    
    
class Placeholder(IdentityNode):
    def __init__(self, name=None):
        self.name = name
        
    def forward(self, graph):
        raise ValueError(f'unbound placeholder {self.name}')
    
    def backward(self, what):
        PlaceholderBackward(self)
    
        
class PlaceholderBackward(IdentityNode):
    def __init__(self, placeholder):
        self.placeholder = placeholder
    
    def forward(self, graph):
        np.zeros_like(_forward(self.placeholder, graph))
    
    def backward(self, what):
        return self
        
    
class Constant(IdentityNode):
    def __init__(self, value, name=None):
        self.value = np.asarray(value)
        self.name = name
    
    def forward(self, graph):
        return self.value
    
    def backward(self, what):
        if self.matches(what):
            return Constant(np.ones_like(self.value))
        else:
            return Constant(np.zeros_like(self.value))

    def matches(self, what):
        if isinstance(what, Constant):
            return self.name == what.name
        
        return self.name == what

class BinaryNode(Node):
    def __init__(self, a, b):
        self.a = a
        self.b = b
    
    def _key(self):
        return self.a, self.b

    
class Addition(BinaryNode):
    def forward(self, graph):
        return _forward(self.a, graph) + _forward(self.b, graph)
        
    def backward(self, what):
        return self.a.backward(what) + self.b.backward(what)

    
class Subtraction(BinaryNode):
    def forward(self, graph):
        return _forward(self.a, graph) - _forward(self.b, graph)
        
    def backward(self, what):
        return self.a.backward(what) - self.b.backward(what)

        
class Multiplication(BinaryNode):
    def forward(self, graph):
        return _forward(self.a, graph) * _forward(self.b, graph)
    
    def backward(self, what):
        return self.a * self.b.backward(what) + self.a.backward(what) * self.b

    
class Division(BinaryNode):
    def forward(self, graph):
        return _forward(self.a, graph) / _forward(self.b, graph)
    
    def backward(self, what):
        return (self.a.backward(what) * self.b - self.a * self.b.backward(what)) / (self.b * self.b)
    
def _forward(obj, graph):
    if obj in graph:
        return graph[obj]
    
    return obj.forward(graph)


def forward(obj, graph=None):
    if graph is None:
        graph = {}
    
    if isinstance(obj, (list, tuple)):
        return type(obj)(forward(item, graph) for item in obj)
    
    elif isinstance(obj, dict):
        return type(obj)(
            (k, forward(v, graph))
            for k, v in obj.items()
        )
    
    else:
        return _forward(obj, graph)

In [203]:
x = Constant(2, name='x')
y = Constant(3, name='y')

g = x / y

print(forward([g, g.backward(x), g.backward(y), g.backward(y).backward(y)]))

[0.66666666666666663, 0.33333333333333331, -0.22222222222222221, 0.14814814814814814]


## The Hessian Matrix

In [205]:
# TODO: implement Hessian computation

## Regularization in Neural Networks

## Mixture Density Networks

## Bayesian Neural Networks